In [15]:
# !pip install xmljson lxml
# !conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
# !pip install transformers
# !conda install gensim numba

In [3]:
import json, xmljson #, xmltodict
import pickle
from lxml.etree import parse, fromstring, tostring
from xmljson import badgerfish as bf
import torch
from transformers import BertModel, BertTokenizer
import pandas as pd, networkx as nx
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
# <?xml version="1.0" encoding="utf-8" standalone="yes"?>
# Please remove fist line and the end sybmol(wtfit) in Posts.xml
with open('../Posts.xml','r',encoding="utf-8-sig") as f:
    xml_raw = f.read()
    data = bf.data(fromstring(xml_raw))

In [18]:
# ld = []

# for i, d in enumerate(data["documents"]["document"]):
#     dd = dict()
#     for key, value in d.items():
#         # print(key, i)
#         if key in ['isRepost', 'TotalVotes', 'PostGeoLocation','Parent', 'RootPostID']:
#             dd[key] = value
#         elif key == "OriginalPost":
#             #TODO not to remove original post
#             pass
#         #TODO make time preprocessing
#         else:
#             try:
#                 dd[key] = value['$']
#             except KeyError:
#                 print("line: ", i, "key: ", key, "value: ", value)
#                 dd[key] = value
#     ld.append(dd)


In [19]:
ld = []

for i, d in enumerate(data["documents"]["document"]):
    dd = dict()
    # Remove empty texts
    try:
        for key, value in d.items():
            # print(key, i)
            if key in ['isRepost', 'TotalVotes', 'PostGeoLocation','Parent', 'RootPostID']:
                dd[key] = value
            elif key == "OriginalPost":
                #TODO not to remove original post
                pass
            #TODO make time preprocessing
            else:
                dd[key] = value['$']
                
    except KeyError:
        continue
    ld.append(dd)

In [20]:
df = pd.DataFrame(ld)

In [21]:
pd.options.display.max_columns = None
df.columns

Index(['_id', 'CrawlId', 'WorkflowId', 'AnalisysId', 'LoadDate', 'Snippets',
       'Text', 'URL', 'CreatedDate', 'PostType', 'ID', 'AuthorID', 'OwnerID',
       'AuthorName', 'LikeCount', 'ULikes', 'CommentCount', 'UComments',
       'ShareCount', 'UShares', 'HashTags', 'References', 'Polls',
       'ViewsCount', 'Objects', 'Rubrics', 'Facts', 'DCluster',
       'DocumentRubrics', 'DocumentTags', 'DocumentClasses', 'wikiPageStat',
       'GISobjects', 'spamWeight', 'Speakers', 'Translations', 'GeoConcepts',
       'Photos', 'Videos', 'Documents', 'Audios', 'Verboten', 'Screenshots',
       'OperDocuments', 'WebPageDumps', 'Annotation', 'Answers', 'Mentions',
       'Summarize', 'MentionedPersons', 'MentionedOrganizations', 'Embeddings',
       'Sentences', 'Sntag', 'isRepost', 'TotalVotes', 'PostGeoLocation',
       'Parent', 'RootPostID'],
      dtype='object')

In [22]:
df.head()

,_id,CrawlId,WorkflowId,AnalisysId,LoadDate,Snippets,Text,URL,CreatedDate,PostType,ID,AuthorID,OwnerID,AuthorName,LikeCount,ULikes,CommentCount,UComments,ShareCount,UShares,HashTags,References,Polls,ViewsCount,Objects,Rubrics,Facts,DCluster,DocumentRubrics,DocumentTags,DocumentClasses,wikiPageStat,GISobjects,spamWeight,Speakers,Translations,GeoConcepts,Photos,Videos,Documents,Audios,Verboten,Screenshots,OperDocuments,WebPageDumps,Annotation,Answers,Mentions,Summarize,MentionedPersons,MentionedOrganizations,Embeddings,Sentences,Sntag,isRepost,TotalVotes,PostGeoLocation,Parent,RootPostID
0,5ee831af6e20701cf124f693,"[""VK-1592275366824292""]",[9000002],null,2020-06-16T02:42:51.348Z,"[{\n ""LoadDate"" : ISODate(""2020-06-16T02:42...",Валентина Пономарева – Искушение (1989)\n#jazz...,https://vk.com/wall-137472625_16131,2020-06-16T00:59:02Z,1,VK_-137472625_16131,VK_-137472625,VK_-137472625,Звуковые волны,0.0,null,0.0,null,1.0,null,null,"[""https://www.discogs.com/%D0%92%D0%B0%D0%BB%D...",null,9.0,null,null,null,null,null,null,null,null,null,100.0,null,null,null,"[{\n ""URL"" : ""https://sun9-62.userapi.com/c...",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,VK,NaN,NaN,NaN,NaN,NaN
1,5ee831af6e20701cf124f694,"[""VK-1592275366824292""]",[9000002],null,2020-06-16T02:42:51.348Z,"[{\n ""LoadDate"" : ISODate(""2020-06-16T02:42...",Требуется копирайтер для сообществ в социальны...,https://vk.com/wall-110896651_54666,2020-06-16T00:58:28Z,1,VK_-110896651_54666,VK_-110896651,VK_-110896651,Kleinburd Projects Ru,0.0,null,0.0,null,0.0,null,null,"[""http://feedproxy.google.com/~r/KleinburdProj...",null,1.0,null,null,null,null,null,null,null,null,null,100.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,VK,NaN,NaN,NaN,NaN,NaN
2,5ee831af6e20701cf124f695,"[""VK-1592275366824292""]",[9000002],null,2020-06-16T02:42:51.348Z,"[{\n ""LoadDate"" : ISODate(""2020-06-16T02:42...","И так, начало разработки проекта положено. Я н...",https://vk.com/wall-196351490_2,2020-06-15T23:29:23Z,1,VK_-196351490_2,VK_-196351490,VK_-196351490,Hardcore in soul | Unturned,0.0,null,0.0,null,0.0,null,null,null,null,1.0,null,null,null,null,null,null,null,null,null,100.0,null,null,null,"[{\n ""URL"" : ""https://sun9-15.userapi.com/Z...",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,VK,NaN,NaN,NaN,NaN,NaN
3,5ee831af6e20701cf124f696,"[""VK-1592275366824292""]",[9000002],null,2020-06-16T02:42:51.348Z,"[{\n ""LoadDate"" : ISODate(""2020-06-16T02:42...",Требуется копирайтер для сообществ в социальны...,https://vk.com/wall-184898993_14912,2020-06-15T22:52:17Z,1,VK_-184898993_14912,VK_-184898993,VK_-184898993,"Заявки, лиды, проекты и задач на рерайт/копирайт",0.0,null,0.0,null,0.0,null,null,"[""https://www.fl.ru/projects/4435285/kopirayte...",null,2.0,null,null,null,null,null,null,null,null,null,100.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,VK,NaN,NaN,NaN,NaN,NaN
4,5ee831af6e20701cf124f698,"[""VK-1592275366824292""]",[9000002],null,2020-06-16T02:42:51.348Z,"[{\n ""LoadDate"" : ISODate(""2020-06-16T02:42...","бро тут делал видосы с войсами, и чет мне тоже...",https://vk.com/wall-164251213_23710,2020-06-15T22:06:11Z,1,VK_-164251213_23710,VK_-164251213,VK_-164251213,babyblade,6.0,null,0.0,null,0.0,null,null,null,null,18.0,null,null,null,null,null,null,null,null,null,100.0,null,null,null,null,"[{\n ""URL"" : ""https://pvv4.vkuservideo.net/...",null,null,null,null,null,null,null,null,null,null,null,null,null,null,VK,NaN,NaN,NaN,NaN,NaN


In [23]:
df.iloc[46738,:10]

_id                                     5ee851c06e20701cf125b754
CrawlId                                  ["VK-1592275366824292"]
WorkflowId                                             [9000002]
AnalisysId                                                  null
LoadDate                                2020-06-16T02:42:51.348Z
Snippets       [{\n    "LoadDate" : ISODate("2020-06-16T02:42...
Text                                                           1
URL                            https://vk.com/wall357686587_5289
CreatedDate                                 2020-06-15T19:01:06Z
PostType                                                       1
Name: 46738, dtype: object

In [24]:
df.drop(46738,inplace=True)

In [25]:
df[['ID', 'AuthorID', 'OwnerID',
     'Text', 'URL', 'CreatedDate', 'PostType','AuthorName', 'LikeCount',
     'ShareCount', 'UShares', 'References', 'ViewsCount',
     'spamWeight', 'Photos', 'Videos', 'Documents',
     'Audios', 'Mentions']].to_csv("../post_df.csv")

In [27]:
# Base text simularity
# code: https://medium.com/better-programming/introduction-to-gensim-calculating-text-similarity-9e8b55de342d
# BERT http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html
# code: 
# https://github.com/huggingface/transformers
# http://docs.deeppavlov.ai/en/master/features/models/bert.html#bert-as-embedder
# rename bert_config.json to config.json
pretrained_weights = "../ru_conversational_cased_L-12_H-768_A-12_pt/"
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)

In [28]:
# text_tok = []
# for i, text in enumerate(df['Text']):
#     try:
#         text_tok.append(tokenizer.encode(text, add_special_tokens=True))
#     except ValueError:
#         print(i, text)
text_tok = [tokenizer.encode(text, add_special_tokens=True) for text in df['Text']]

In [30]:
# with open('../text_tok.pkl', 'wb') as f:
#     pickle.dump(obj=text_tok, file=f)

In [1]:
import json, xmljson #, xmltodict
import pickle
from lxml.etree import parse, fromstring, tostring
from xmljson import badgerfish as bf
import torch
from transformers import BertModel, BertTokenizer
import pandas as pd, networkx as nx
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
with open('../text_tok.pkl', 'rb') as f:
    text_tok = pickle.load(file=f)

In [3]:
def it(text):
    return text

tfidf = TfidfVectorizer(tokenizer=it, lowercase=False)    
tfidf_tok = tfidf.fit_transform(text_tok)

In [4]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.spatial.distance import cosine
from tqdm.notebook import tqdm
from numba import njit, prange
import numpy as np

In [5]:
tfidf_tok_q = tfidf_tok.astype(dtype = np.float32, casting='unsafe', copy=True)

In [6]:
del text_tok, tfidf_tok

In [7]:
# del df, data, ld, tfidf_tok

In [8]:
# n = tfidf_tok_q.shape[0]
# sim_matrix = np.empty(shape=(n,n))

# for i, row in tqdm(enumerate(tfidf_tok_q)):
#     for j, col in tqdm(enumerate(tfidf_tok_q)):
#         # print(i, j)
#         sim_matrix[i, j] = cosine(u=row.toarray(), v=col.toarray())

In [9]:
tfidf_tok_q = tfidf_tok_q.toarray()

In [13]:
def cosine_sim(arr):
    n = arr.shape[0]
    sim_matrix = np.empty(shape=(n,n))
    @njit(parallel=True)
    def in_cycle(row):
        sim_row = np.zeros(shape=n)
        for j in prange(n):
            col = arr[j, :]
            uv = row * col
            uu = row * row
            vv = col * col
            sim_row[j] = uv.sum()/np.sqrt(uu.sum()*vv.sum())
        return sim_row
    for i in tqdm(range(n)):
        #print(i)
        row = arr[i, :]
        sim_matrix[i, :] = np.array(in_cycle(row))
    return sim_matrix

In [14]:
#sim_matrix = tfidf_tok_q.dot(tfidf_tok_q.T)
from sklearn.metrics.pairwise import cosine_similarity
# similarities = cosine_similarity(tfidf_tok_q)
sim_matrix = cosine_sim(arr=tfidf_tok_q)

KeyboardInterrupt: 

In [10]:
sim_matrix = tfidf_tok_q.dot(tfidf_tok_q.T)
# from sklearn.metrics.pairwise import cosine_similarity
# similarities = cosine_similarity(tfidf_tok_q)
# sim_matrix = cosine_sim(arr=tfidf_tok_q)

In [11]:
del tfidf_tok_q

In [15]:
sim_matrix

array([[0.99999994, 0.10161201, 0.17042074, ..., 0.03096257, 0.02170495,
        0.07290123],
       [0.10161201, 0.99999994, 0.12352774, ..., 0.02148255, 0.01787589,
        0.0469254 ],
       [0.17042074, 0.12352774, 0.99999994, ..., 0.05742138, 0.02690176,
        0.05902331],
       ...,
       [0.03096257, 0.02148255, 0.05742138, ..., 1.        , 0.00722093,
        0.02417378],
       [0.02170495, 0.01787589, 0.02690176, ..., 0.00722093, 0.99999994,
        0.01253517],
       [0.07290123, 0.0469254 , 0.05902331, ..., 0.02417378, 0.01253517,
        1.0000001 ]], dtype=float32)

In [12]:
# np.save('../sim_matrix', sim_matrix, allow_pickle=True, fix_imports=True)

In [13]:
sim_matrix_q = sim_matrix.astype(np.float16)

In [14]:
# np.save('../sim_matrix_q', sim_matrix_q, allow_pickle=True, fix_imports=True)

In [ ]:
# with open('../sim_matrix.pkl', 'wb') as f:
#     pickle.dump(obj=sim_matrix, file=f)

In [ ]:
sim_matrix_q = np.load('../sim_matrix_q')

In [ ]:
sim_query = gensim.similarities.SparseMatrixSimilarity(corpus=tfidf_tok, num_features=tfidf_tok.shape[1])

In [ ]:

# Encode text
input_ids = torch.tensor([tokenizer.encode("Here is some text to encode", add_special_tokens=True)])  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
with torch.no_grad():
    last_hidden_states = model(input_ids)[0]  # Models outputs are now tuples

# Each architecture is provided with several class for fine-tuning on down-stream tasks, e.g.
BERT_MODEL_CLASSES = [BertModel, BertForPreTraining, BertForMaskedLM, BertForNextSentencePrediction,
                      BertForSequenceClassification, BertForTokenClassification, BertForQuestionAnswering]

# All the classes for an architecture can be initiated from pretrained weights for this architecture
# Note that additional weights added for fine-tuning are only initialized
# and need to be trained on the down-stream task
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

for model_class in BERT_MODEL_CLASSES:
    # Load pretrained model/tokenizer
    model = model_class.from_pretrained(pretrained_weights)

    # Models can return full list of hidden-states & attentions weights at each layer
    model = model_class.from_pretrained(pretrained_weights,
                                        output_hidden_states=True,
                                        output_attentions=True)
    input_ids = torch.tensor([tokenizer.encode("Let's see all hidden-states and attentions on this text")])
    all_hidden_states, all_attentions = model(input_ids)[-2:]

    # Models are compatible with Torchscript
    model = model_class.from_pretrained(pretrained_weights, torchscript=True)
    traced_model = torch.jit.trace(model, (input_ids,))

    # Simple serialization for models and tokenizers
    model.save_pretrained('./directory/to/save/')  # save
    model = model_class.from_pretrained('./directory/to/save/')  # re-load
    tokenizer.save_pretrained('./directory/to/save/')  # save
    tokenizer = BertTokenizer.from_pretrained('./directory/to/save/')  # re-load

    # SOTA examples for GLUE, SQUAD, text generation...

